# 거리기반 제외

## DB연결해서 불러오기

In [ ]:
import cx_Oracle
import pandas as pd
connection = cx_Oracle.connect("party/party@192.168.30.240:1521/xe")

query = """select * from PlACEINFO"""

df = pd.read_sql(query, con=connection)

In [ ]:
place_df=pd.read_sql(query, con=connection)

In [ ]:
place_df.head(5)

In [ ]:
    place_df.sort_values('ID', ascending=True)

## 쓸 컬럼 뽑아내기

In [ ]:
data=place_df[['카테고리','제목','주소','키워드리스트']]

In [ ]:
data_sort=data

In [ ]:
data_sort.to_csv('추천알고리즘참고용.csv')

In [ ]:
data2=pd.read_csv('추천알고리즘참고용.csv')

In [ ]:
data2

In [ ]:
data_rename=data2.rename(columns={'Unnamed: 0':'id'})

In [ ]:
data_rename

In [ ]:
data_drop=data_rename.drop('id', axis=1, inplace=True)

In [ ]:
data_rename

## 추천알고리즘1 -실패

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 데이터프레임에서 가져오기
data_recommend = data_rename

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# CountVectorizer를 사용하여 텍스트 데이터 벡터화
count_vectorizer = CountVectorizer(ngram_range=(1, 3))
c_vector_combined = count_vectorizer.fit_transform(data_for_recommend['combined'])

# 코사인 유사도 계산
similarity_combined = cosine_similarity(c_vector_combined, c_vector_combined).argsort()[:,::-1]

# 특정 장소를 기반으로 영화 추천
def recommend_place_list(df, 추천장소, top=10):
    # 특정 장소 정보 뽑아내기
    target_place_index = df[df['제목'] == 추천장소].index.values
    
    # 타겟 장소와 비슷한 코사인 유사도 값
    sim_index = similarity_combined[target_place_index,:top].reshape(-1)
    # 본인 제외
    sim_index = sim_index[sim_index != target_place_index]
    
    # 추천 결과 새로운 데이터프레임 생성
    result = df.iloc[sim_index]
    
    return result

In [ ]:
# 추천 알고리즘 실행
recommendations = recommend_movie_list(data_recommend, 추천장소='강화 석수문')
print(recommendations[['카테고리', '제목']])

## 추천알고리즘2 - 성공(거리기반 일단 제외시킴, 키워드만검색시 뜨는것도 일단됨) 

In [74]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 데이터프레임에서 가져오기
data_recommend = data_rename

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# CountVectorizer를 사용하여 텍스트 데이터 벡터화
count_vectorizer = CountVectorizer(ngram_range=(1, 3))
c_vector_combined = count_vectorizer.fit_transform(data_for_recommend['combined'])

# 코사인 유사도 계산
similarity_combined = cosine_similarity(c_vector_combined, c_vector_combined).argsort()[:,::-1]

# 특정 카테고리와 키워드에 맞는 추천 알고리즘 함수
def recommend_titles(df, 카테고리, 키워드, top_n=3):
    # 특정 카테고리와 키워드에 맞는 데이터 추출
    filtered_data = df[(df['카테고리'] == 카테고리) & (df['키워드리스트'].str.contains(키워드))]
    
    # 추천 결과 새로운 데이터프레임 생성, '키워드'의 숫자로 정렬
    recommended_data = filtered_data.sort_values('키워드리스트', ascending=False)[:top_n]
    
    return recommended_data[['카테고리', '제목']]

# 특정 카테고리와 키워드를 기반으로 영화 추천 실행
추천카테고리 = '숙박'
추천키워드 = '넷플릭스'
top_n = 3
recommended_titles = recommend_titles(data_recommend, 추천카테고리, 추천키워드, top_n)
print(recommended_titles)

NameError: name 'data_rename' is not defined

# 거리기반 추가하기

## DB연결해서 불러오기

In [2]:
import pandas as pd
import cx_Oracle
import pandas as pdb
connection = cx_Oracle.connect("party/party@192.168.30.240:1521/xe")

query = """select * from PlACEINFO"""

place_df = pd.read_sql(query, con=connection)

C:\Users\tj-bu\AppData\Local\Temp\ipykernel_5332\4165351655.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  place_df = pd.read_sql(query, con=connection)


In [3]:
place_df.head(5)

,ID,카테고리,소분류,제목,주소,전화번호,경도,위도,애완동물동반가능정보,주차시설,...,판매품목,편의시설,대표메뉴,포장가능여부,입실시간,퇴실시간,부대시설,키워드리스트,이미지주소1,이미지주소2
0,2,관광,역사문화,강화 석수문,인천 강화군 강화읍 고비고개로19번길 12,032-930-4571,126.474572,37.745366,"가능 (단, 개목줄 착용, 배설물 처리를 위한 비닐봉지 등을 구비한 이용객에 한하여...",있음 (무료),...,None,None,None,None,None,None,None,"강화/154,수문/122,강화산성/113,서문/92,강화도/65,강화읍/49,벚꽃/...",http://post.phinf.naver.net/MjAxOTExMjdfMjU1/M...,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...
1,4,관광,역사문화,강화 전등사,인천 강화군 길상면 전등사로 37-41,032-937-0125,126.484406,37.632160,불가(전등사 관람은 목줄과 함께 가능),있음 (유료),...,None,None,None,None,None,None,None,"전등사/376,강화/163,강화도/69,삼랑성/66,대웅보전/47,대웅전/46,주차...",http://imgnews.naver.net/image/001/2020/09/24/...,http://imgnews.naver.net/image/5157/2014/12/16...
2,10,관광,기타,애관극장,인천 중구 개항로 63-2,032-761-7177,126.629807,37.472359,불가,있음,...,None,None,None,None,None,None,None,"극장/230,애관/184,영화/153,인천/115,영화관/66,상영/47,역사/38...",http://imgnews.naver.net/image/5154/2022/04/18...,http://imgnews.naver.net/image/028/2021/11/02/...
3,13,관광,역사문화,강화 보문사,인천 강화군 삼산면 삼산남로828번길 44,032-933-8271~3,126.321015,37.688689,불가,있음 (유료),...,None,None,None,None,None,None,None,"보문사/312,강화/157,석모도/114,관음/69,바다/69,기도/66,계단/58...",http://post.phinf.naver.net/MjAxODAxMTdfMTk5/M...,https://t1.daumcdn.net/cafeattach/1QaS9/cdf1a3...
4,17,관광,체험관광,강화 화문석마을,인천 강화군 송해면 전망대로 452,"032-934-2290,6245",126.458513,37.799880,불가,있음 (무료),...,None,None,None,None,None,None,None,"화문석/299,마을/158,체험/141,강화/139,강화도/57,왕골/55,만들기/...",https://dbscthumb-phinf.pstatic.net/4904_000_1...,https://dbscthumb-phinf.pstatic.net/4749_000_1...


## 쓸컬럼 뽑아오기

In [4]:
data=place_df[['카테고리','제목','주소','위도','경도','키워드리스트']]

In [5]:
data.tail(5)

,카테고리,제목,주소,위도,경도,키워드리스트
879,숙박,호텔원티드,인천 서구 북항로32번길 20,37.508314,126.645278,"호텔/463,원티드/250,인천/230,청라/141,객실/110,호캉스/107,이용..."
880,숙박,알로하호텔,인천 남동구 남동대로777번길 32,37.452192,126.706061,"호텔/412,알로하/285,하남/271,스타/125,객실/114,필드/107,이용/..."
881,숙박,알프스모텔,인천 서구 가정로261번길 15,37.508106,126.670595,"모텔/185,알프스/144,온천/79,숙소/49,가족/49,울산/39,위치/34,호..."
882,숙박,에이스모텔,인천 미추홀구 아암대로53번길 15,37.456330,126.636301,"모텔/140,에이스/112,숙소/89,해남/72,여행/34,아야/31,해변/29,위..."
883,숙박,영종호텔,인천 중구 운중로21번길 8,37.494245,126.536712,"호텔/512,그레이스/236,영종/231,영종도/149,오션/142,인천/103,체..."


## 추천알고리즘 계산하기

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import math

# 데이터프레임에서 가져오기
data_recommend = data

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# CountVectorizer를 사용하여 텍스트 데이터 벡터화
count_vectorizer = CountVectorizer(ngram_range=(1, 3))
c_vector_combined = count_vectorizer.fit_transform(data_for_recommend['combined'])

# 코사인 유사도 계산
similarity_combined = cosine_similarity(c_vector_combined, c_vector_combined).argsort()[:,::-1]

# 위도 경도를 기반으로 두 지점 간의 거리를 계산하는 함수
def haversine_distance(lat1, lon1, lat2, lon2):
    radius = 6371.0  # 지구의 반지름 (단위: km)

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c
    return distance

# 특정 카테고리와 키워드에 맞는 추천 알고리즘 함수
def recommend_titles_with_distance(df, 카테고리, 키워드, top_n=3):
    # 특정 카테고리와 키워드에 맞는 데이터 추출
    filtered_data = df[(df['카테고리'] == 카테고리) & (df['키워드리스트'].str.contains(키워드))]
    
    # 추천 결과를 담을 리스트
    recommended_data_list = []
    
    # 거리 계산 및 추천 결과 생성
    for index, row in filtered_data.iterrows():
        lat1, lon1 = row['위도'], row['경도']
        distances = []
        for idx, place_row in data_recommend.iterrows():
            lat2, lon2 = place_row['위도'], place_row['경도']
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            if distance != 0 :
                distances.append((place_row['제목'], distance))
            else :
                pass
        distances.sort(key=lambda x: x[1])
        recommended_data_list.append([row['카테고리'], row['제목'], distances[:top_n]])
    
    return recommended_data_list

# 특정 카테고리와 키워드를 기반으로 영화 추천 및 거리 계산 실행
추천카테고리 = '관광'
추천키워드 = '인천'
top_n = 3
recommended_titles_with_distance = recommend_titles_with_distance(data_recommend, 추천카테고리, 추천키워드, top_n)
for item in recommended_titles_with_distance[:3]:
    카테고리, 제목, 거리정보 = item
    print(f"카테고리: {카테고리}, 제목: {제목}")
    for place, distance in 거리정보:
        print(f"  가까운 장소: {place}, 거리: {distance:.2f} km")

카테고리: 관광, 제목: 강화 석수문
  가까운 장소: 국화호수, 거리: 0.79 km
  가까운 장소: 강화나들길, 거리: 0.87 km
  가까운 장소: 용흥궁식당, 거리: 0.95 km
카테고리: 관광, 제목: 강화 전등사
  가까운 장소: 프렌쥬랜드, 거리: 0.78 km
  가까운 장소: 강화 온수리 성공회성당, 거리: 1.04 km
  가까운 장소: 강화루지(강화씨사이드리조트), 거리: 2.06 km
카테고리: 관광, 제목: 애관극장
  가까운 장소: 돈비어천가, 거리: 0.11 km
  가까운 장소: 인천 답동성당, 거리: 0.11 km
  가까운 장소: 스카이모텔, 거리: 0.12 km


## 키워드 검색없을때(3개 완-전체 수집된 자료로 해봐야 제대로 나올거같음) 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import math

top_n = 3

# 데이터프레임에서 가져오기 (예시 데이터로 대체)
# 데이터프레임에서 가져오기
data_recommend = data

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트', '위도', '경도']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# CountVectorizer를 사용하여 텍스트 데이터 벡터화
count_vectorizer = CountVectorizer(ngram_range=(1, 3))
c_vector_combined = count_vectorizer.fit_transform(data_for_recommend['combined'])

# 코사인 유사도 계산
similarity_combined = cosine_similarity(c_vector_combined, c_vector_combined).argsort()[:,::-1]

# 위도 경도를 기반으로 두 지점 간의 거리를 계산하는 함수
def haversine_distance(lat1, lon1, lat2, lon2):
    radius = 6371.0  # 지구의 반지름 (단위: km)
    
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c
    return distance if not math.isnan(distance) else None
# 특정 카테고리와 키워드에 맞는 추천 알고리즘 함수

def recommend_titles_with_distance(df, 추천카테고리=None, 추천키워드=None, top_n=3):
    filtered_data = df.copy()
    if 추천카테고리:
        filtered_data = filtered_data[filtered_data['카테고리'] == 추천카테고리]
    if 추천키워드:
        filtered_data = filtered_data[filtered_data['키워드리스트'].str.contains(추천키워드)]

    recommended_data_list = []

    for index, row in filtered_data.iterrows():
        lat1, lon1 = row['위도'], row['경도']
        distances = []
        for idx, place_row in data_recommend.iterrows():
            if row['제목'] != place_row['제목']:
                lat2, lon2 = place_row['위도'], place_row['경도']
                distance = haversine_distance(lat1, lon1, lat2, lon2)
                if distance is not None:
                    distances.append((place_row['제목'], distance))
        distances.sort(key=lambda x: x[1])
        recommended_data_list.append([row['카테고리'], row['제목'], distances[:3]])

    return recommended_data_list

# 특정 카테고리와 키워드를 기반으로 영화 추천 및 거리 계산 실행
추천카테고리 = None  # None으로 두면 카테고리를 사용하지 않음
추천키워드 = None  # None으로 두면 키워드를 사용하지 않음

recommended_titles_with_distance = recommend_titles_with_distance(data_recommend, 추천카테고리, 추천키워드, top_n)
for item in recommended_titles_with_distance[:3]:
    카테고리, 제목, 거리정보 = item
    print(f"카테고리: {카테고리}, 제목: {제목}")
    for place, distance in 거리정보:
        print(f"  가까운 장소: {place}, 거리: {distance:.2f} km")

## 위 코드에서 검색어 입력했을떄(3개나옴- 전체 수집된자료로 봐야 제대로 나올거같음)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import math

# 데이터프레임에서 가져오기 (예시 데이터로 대체)
# 데이터프레임에서 가져오기
data_recommend = data

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트', '위도', '경도']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# CountVectorizer를 사용하여 텍스트 데이터 벡터화
count_vectorizer = CountVectorizer(ngram_range=(1, 3)) ## 토근별 빈도수게산 
c_vector_combined = count_vectorizer.fit_transform(data_for_recommend['combined']) #

# 코사인 유사도 계산
similarity_combined = cosine_similarity(c_vector_combined, c_vector_combined).argsort()[:,::-1]

# 위도 경도를 기반으로 두 지점 간의 거리를 계산하는 함수
def haversine_distance(lat1, lon1, lat2, lon2):
    radius = 6371.0  # 지구의 반지름 (단위: km)
    
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c
    return distance if not math.isnan(distance) else None
# 특정 카테고리와 키워드에 맞는 추천 알고리즘 함수
def recommend_titles_with_distance(df, 추천카테고리=None, 추천키워드=None, top_n=3):
    filtered_data = df.copy()
    if 추천카테고리:
        filtered_data = filtered_data[filtered_data['카테고리'] == 추천카테고리]
    if 추천키워드:
        # NaN 값을 빈 문자열로 대체한 후에 키워드를 포함하는지 확인
        filtered_data = filtered_data[filtered_data['키워드리스트'].fillna('').str.contains(추천키워드)]

    recommended_data_list = []

    for index, row in filtered_data.iterrows():
        lat1, lon1 = row['위도'], row['경도']
        distances = []
        for idx, place_row in data_recommend.iterrows():
            if row['제목'] != place_row['제목']:
                lat2, lon2 = place_row['위도'], place_row['경도']
                distance = haversine_distance(lat1, lon1, lat2, lon2)
                if distance is not None:
                    distances.append((place_row['제목'], distance))
        distances.sort(key=lambda x: x[1])
        recommended_data_list.append([row['카테고리'], row['제목'], distances[:top_n]])

    return recommended_data_list

# 특정 카테고리와 키워드를 기반으로 영화 추천 및 거리 계산 실행
추천카테고리 = '강화'  # None으로 두면 카테고리를 사용하지 않음
추천키워드 = '맛집'  # None으로 두면 키워드를 사용하지 않음
top_n = 3
recommended_titles_with_distance = recommend_titles_with_distance(data_recommend, 추천카테고리, 추천키워드, top_n)
for item in recommended_titles_with_distance[:3]:
    카테고리, 제목, 거리정보 = item
    print(f"카테고리: {카테고리}, 제목: {제목}")
    for place, distance in 거리정보:
        print(f"  가까운 장소: {place}, 거리: {distance:.2f} km")

## 키워드리스트 뜨는버전

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import math

# 데이터프레임에서 가져오기 (예시 데이터로 대체)
# 데이터프레임에서 가져오기
data_recommend = data

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트', '위도', '경도']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체(혹시나 있을수도 있는 null값 전처리)
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기 카테고리와 키워드리스트 합병
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# CountVectorizer를 사용하여 텍스트 데이터 벡터화
count_vectorizer = CountVectorizer(ngram_range=(1, 3))
c_vector_combined = count_vectorizer.fit_transform(data_for_recommend['combined'])

# 코사인 유사도 계산
similarity_combined = cosine_similarity(c_vector_combined, c_vector_combined).argsort()[:,::-1]

# 위도 경도를 기반으로 두 지점 간의 거리를 계산하는 함수
def haversine_distance(lat1, lon1, lat2, lon2):
    radius = 6371.0  # 지구의 반지름 (단위: km)
    #위도. 경도를 radian변환
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    #장소1과 장소2의 위도 경도 차이
    dlat = lat2 - lat1
    dlon = lon2 - lon1
     
    #위도 경도 차이로 거리를 계산하는 공식
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c
    return distance if not math.isnan(distance) else None

# 특정 카테고리와 키워드에 맞는 추천 알고리즘 함수, None으로 해야 디폴트값을 쓸수있음
def recommend_titles_with_distance(df, 추천카테고리=None, 추천키워드=None, top_n=3):
    filtered_data = df.copy() 
    if 추천카테고리:
        filtered_data = filtered_data[filtered_data['카테고리'] == 추천카테고리]
    if 추천키워드:
        # NaN 값을 빈 문자열로 대체한 후에 키워드를 포함하는지 확인
        filtered_data = filtered_data[filtered_data['키워드리스트'].fillna('').str.contains(추천키워드)]

    recommended_data_list = []

    for index, row in filtered_data.iterrows():
        lat1, lon1 = row['위도'], row['경도']
        distances = []
        for idx, place_row in data_recommend.iterrows():
            if row['제목'] != place_row['제목']:
                lat2, lon2 = place_row['위도'], place_row['경도']
                distance = haversine_distance(lat1, lon1, lat2, lon2)
                if distance is not None:
                    distances.append((place_row['제목'], distance))
        distances.sort(key=lambda x: x[1])
        recommended_data_list.append([row['카테고리'], row['제목'], row['키워드리스트'], distances[:top_n]])

    return recommended_data_list

# 특정 카테고리와 키워드를 기반으로 제목(장소) 추천 및 거리 계산 실행
추천카테고리 = '숙박'  # None으로 두면 카테고리를 사용하지 않음
추천키워드 = '맛집'  # None으로 두면 키워드를 사용하지 않음
top_n = 3
recommended_titles_with_distance = recommend_titles_with_distance(data_recommend, 추천카테고리, 추천키워드, top_n)

for item in recommended_titles_with_distance[:3]:
    카테고리, 제목, 키워드리스트, 거리정보 = item
    print(f"키워드리스트: {data_recommend[data_recommend['제목'] == 제목]['키워드리스트'].values[0][:30]}, 제목: {제목}")
    for place, distance in 거리정보:
        print(f" 키워드리스트:{키워드리스트[0:30]}, 가까운 장소: {place}, 거리: {distance:.2f} km")

키워드리스트: 호텔/211,베트남/81,레전드/78,하노이/72,조식, 제목: 레전드호텔
 키워드리스트:호텔/211,베트남/81,레전드/78,하노이/72,조식, 가까운 장소: 제우메스호텔, 거리: 0.43 km
 키워드리스트:호텔/211,베트남/81,레전드/78,하노이/72,조식, 가까운 장소: 데이즈호텔&스위트바이윈덤인천에어포트, 거리: 0.46 km
 키워드리스트:호텔/211,베트남/81,레전드/78,하노이/72,조식, 가까운 장소: 하워드 존슨 인천 에어포트, 거리: 0.46 km
키워드리스트: 모텔/113,위드/98,숙소/58,여행/54,울산/44, 제목: 위드모텔
 키워드리스트:모텔/113,위드/98,숙소/58,여행/54,울산/44, 가까운 장소: 초이스모텔, 거리: 0.06 km
 키워드리스트:모텔/113,위드/98,숙소/58,여행/54,울산/44, 가까운 장소: 시골쌈밥, 거리: 0.15 km
 키워드리스트:모텔/113,위드/98,숙소/58,여행/54,울산/44, 가까운 장소: 타카이타이, 거리: 0.17 km
키워드리스트: 호텔/346,인천/345,소울/237,계양/124,계양, 제목: 소울하다계양
 키워드리스트:호텔/346,인천/345,소울/237,계양/124,계양, 가까운 장소: 원할머니보쌈 계산점, 거리: 0.06 km
 키워드리스트:호텔/346,인천/345,소울/237,계양/124,계양, 가까운 장소: 굴축제, 거리: 0.06 km
 키워드리스트:호텔/346,인천/345,소울/237,계양/124,계양, 가까운 장소: 등촌샤브칼국수 계양점, 거리: 0.08 km


# 주소까지 입력

## 주소까지 입력하는 버전

In [65]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import math

# 데이터프레임에서 가져오기 (예시 데이터로 대체)
# 데이터프레임에서 가져오기
data_recommend = data

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트', '위도', '경도']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# CountVectorizer를 사용하여 텍스트 데이터 벡터화
count_vectorizer = CountVectorizer(ngram_range=(1, 3)) ## 토근별 빈도수게산 
c_vector_combined = count_vectorizer.fit_transform(data_for_recommend['combined']) #

# 코사인 유사도 계산
similarity_combined = cosine_similarity(c_vector_combined, c_vector_combined).argsort()[:,::-1]

# 위도 경도를 기반으로 두 지점 간의 거리를 계산하는 함수
def haversine_distance(lat1, lon1, lat2, lon2):
    radius = 6371.0  # 지구의 반지름 (단위: km)
    
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c
    return distance if not math.isnan(distance) else None

# 특정 카테고리와 키워드에 맞는 추천 알고리즘 함수
def recommend_titles_with_distance(df, 추천카테고리=None, 추천키워드=None, 주소=None, top_n=3):
    filtered_data = df.copy()
    if 추천카테고리:
        filtered_data = filtered_data[filtered_data['카테고리'] == 추천카테고리]
    if 추천키워드:
        # NaN 값을 빈 문자열로 대체한 후에 키워드를 포함하는지 확인
        filtered_data = filtered_data[filtered_data['키워드리스트'].fillna('').str.contains(추천키워드)]
    if 주소:
        # 주소에서 구나 군까지만 추출하여 필터링
        filtered_data = filtered_data[filtered_data['주소'].str.contains(주소)]

    recommended_data_list = []

    for index, row in filtered_data.iterrows():
        lat1, lon1 = row['위도'], row['경도']
        distances = []
        for idx, place_row in data_recommend.iterrows():
            if row['제목'] != place_row['제목']:
                lat2, lon2 = place_row['위도'], place_row['경도']
                distance = haversine_distance(lat1, lon1, lat2, lon2)
                if distance is not None:
                    distances.append((place_row['제목'], distance))
        distances.sort(key=lambda x: x[1])
        recommended_data_list.append([row['카테고리'], row['제목'], distances[:top_n]])

    return recommended_data_list

# 특정 카테고리와 키워드, 주소를 기반으로 영화 추천 및 거리 계산 실행
추천카테고리 = '' # None으로 두면 카테고리를 사용하지 않음
추천키워드 = '맛집'  # None으로 두면 키워드를 사용하지 않음
주소 = '부평'  # None으로 두면 주소를 사용하지 않음
top_n = 3
recommended_titles_with_distance = recommend_titles_with_distance(data_recommend, 추천카테고리, 추천키워드, 주소, top_n)

for item in recommended_titles_with_distance[:3]:
    카테고리, 제목, 거리정보 = item
    print(f"카테고리: {카테고리}, 제목: {제목}")
    for place, distance in 거리정보:
        print(f"  가까운 장소: {place}, 거리: {distance:.2f} km")

카테고리: 쇼핑, 제목: 부평깡시장
  가까운 장소: 진흥종합시장, 거리: 0.13 km
  가까운 장소: 부평종합시장, 거리: 0.15 km
  가까운 장소: 부평종합시장 식품위생우수업소, 거리: 0.17 km
카테고리: 쇼핑, 제목: 부평문화의거리
  가까운 장소: 평리단길, 거리: 0.14 km
  가까운 장소: 라임호텔, 거리: 0.21 km
  가까운 장소: 버니, 거리: 0.23 km
카테고리: 쇼핑, 제목: 열우물전통시장(구 십정종합시장)
  가까운 장소: 동암아구해물탕, 거리: 0.53 km
  가까운 장소: 와! 참치다, 거리: 0.54 km
  가까운 장소: 함경꿩만두 칼국수 함경 닭도리, 거리: 0.57 km


## 주소입력받고 주소까지 출력하게끔

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import math

# 데이터프레임에서 가져오기 (예시 데이터로 대체)
# 데이터프레임에서 가져오기
data_recommend = data

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트', '위도', '경도', '주소']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# CountVectorizer를 사용하여 텍스트 데이터 벡터화
count_vectorizer = CountVectorizer(ngram_range=(1, 3))
c_vector_combined = count_vectorizer.fit_transform(data_for_recommend['combined'])

# 코사인 유사도 계산
similarity_combined = cosine_similarity(c_vector_combined, c_vector_combined).argsort()[:,::-1]

# 위도 경도를 기반으로 두 지점 간의 거리를 계산하는 함수
def haversine_distance(lat1, lon1, lat2, lon2):
    radius = 6371.0  # 지구의 반지름 (단위: km)

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c
    return distance if not math.isnan(distance) else None

# 특정 카테고리와 키워드, 주소를 기반으로 영화 추천 및 거리 계산 실행
def recommend_titles_with_distance(df, 추천카테고리=None, 추천키워드=None, 추천주소=None, top_n=3):
    filtered_data = df.copy()
    if 추천카테고리:
        filtered_data = filtered_data[filtered_data['카테고리'] == 추천카테고리]
    if 추천키워드:
        # NaN 값을 빈 문자열로 대체한 후에 키워드를 포함하는지 확인
        filtered_data = filtered_data[filtered_data['키워드리스트'].fillna('').str.contains(추천키워드)]
    if 추천주소:
        # 주소에서 구나 군까지만 추출하여 필터링
        filtered_data = filtered_data[filtered_data['주소'].str.contains(추천주소)]

    recommended_data_list = []

    for index, row in filtered_data.iterrows():
        lat1, lon1 = row['위도'], row['경도']
        distances = []
        for idx, place_row in data_recommend.iterrows():
            if row['제목'] != place_row['제목']:
                lat2, lon2 = place_row['위도'], place_row['경도']
                distance = haversine_distance(lat1, lon1, lat2, lon2)
                if distance is not None:
                    distances.append((place_row['제목'], place_row['주소'], distance))
        distances.sort(key=lambda x: x[2])
        recommended_data_list.append([row['카테고리'], row['제목'], distances[:top_n]])

    return recommended_data_list

# 특정 카테고리와 키워드, 주소를 기반으로 영화 추천 및 거리 계산 실행
추천카테고리 = ''  # None으로 두면 카테고리를 사용하지 않음
추천키워드 = '맛집'  # None으로 두면 키워드를 사용하지 않음
추천주소 = '부평'  # None으로 두면 주소를 사용하지 않음
top_n = 3
recommended_titles_with_distance = recommend_titles_with_distance(data_recommend, 추천카테고리, 추천키워드, 추천주소, top_n)

for item in recommended_titles_with_distance[:3]:
    카테고리, 제목, 거리정보 = item
    print(f"카테고리: {카테고리}, 제목: {제목}")
    for place, address, distance in 거리정보:
        print(f"  가까운 장소: {place}, 주소: {address}, 거리: {distance:.2f} km")

카테고리: 쇼핑, 제목: 부평깡시장
  가까운 장소: 진흥종합시장, 주소: 인천 부평구 부흥로304번길 27 진흥종합시장, 거리: 0.13 km
  가까운 장소: 부평종합시장, 주소: 인천 부평구 부흥로316번길 38-3, 거리: 0.15 km
  가까운 장소: 부평종합시장 식품위생우수업소, 주소: 인천 부평구 부흥로 316번길 38-3, 거리: 0.17 km
카테고리: 쇼핑, 제목: 부평문화의거리
  가까운 장소: 평리단길, 주소: 인천 부평구 부평문화로 65, 거리: 0.14 km
  가까운 장소: 라임호텔, 주소: 인천 부평구 부평문화로79번길 40, 거리: 0.21 km
  가까운 장소: 버니, 주소: 인천 부평구 경원대로1417번길 17-6, 거리: 0.23 km
카테고리: 쇼핑, 제목: 열우물전통시장(구 십정종합시장)
  가까운 장소: 동암아구해물탕, 주소: 인천 부평구 백범로468번길 45, 거리: 0.53 km
  가까운 장소: 와! 참치다, 주소: 인천 부평구 백범로468번길 41-1, 거리: 0.54 km
  가까운 장소: 함경꿩만두 칼국수 함경 닭도리, 주소: 인천 부평구 백범로468번길 32, 거리: 0.57 km


## TF-IDF사용(키워드제외, 한줄한줄 주석있음)

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import math
## 라이브러리 임포트

# 데이터프레임에서 가져오기
data_recommend = data

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트', '위도', '경도', '주소']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체(전처리)
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기(분석할 텍스트 카테고리+키워드리스트)
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3)) # 텍스트데이터를 TF_IDF 행렬로 변환 ngram_range 파라미터를 통해서 구성 범위 설정
tfidf_matrix = tfidf_vectorizer.fit_transform(data_for_recommend['combined'])

# 코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 
# TF_IDF 행렬을 사용하여 코사인 유사도를 계산한다. 각 제목(장소) 간의 유사도(카테고리+키워드리스트 분석)를 나타내는 행렬

# 위도 경도를 기반으로 두 지점 간의 거리를 계산하는 함수
def haversine_distance(lat1, lon1, lat2, lon2):
    radius = 6371.0  # 지구의 반지름 (단위: km)

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c
    return distance if not math.isnan(distance) else None
#피타고라스의 정리와 유사함 a^2+c^2=b^2

# 특정 카테고리와 키워드, 주소를 기반으로 제목(장소) 추천 및 거리 계산 실행
def recommend_distance(df, 추천카테고리=None, 추천키워드=None, 추천주소=None, top_n=3):
    #DB의 데이터프레임을 받아서 특정 조건에 맞는 제목(장소)를 추천하고 그 제목(장소)들의 거리 정보를 함께 반환한다.
    #df=Db의 데이터프레임, 추천카테고리 : Df의 '카테고리', 추천키워드 : DF의 '키워드리스트', 추천주소 : Df의 제목(장소)의 주소 top_n: 개수
    filtered_data = df.copy()
    #df를 복사하여 filtererd_data 변수에 할당시킴 추천 조건을 적용하여 필터링한 데이터를 생성하면서, 원본데이터프레임은 그대로 유지
    if 추천카테고리:
        filtered_data = filtered_data[filtered_data['카테고리'] == 추천카테고리]
        #추천카테고리가 입력한 경우 (None이 아닌 경우), filtered_data를 해당 카테고리에 맞는 제목(장소)들로 필터링.
    if 추천키워드:
        # NaN 값을 빈 문자열로 대체한 후에 키워드를 포함하는지 확인
        filtered_data = filtered_data[filtered_data['키워드리스트'].fillna('').str.contains(추천키워드)]
        # 만약 추천키워드가 주어진 경우 (None이 아닌 경우), filtered_data를 해당 키워드가 포함된 제목들로 필터링
        # 이때 fillna('')를 사용하여 키워드가 NaN인 경우에도 필터링할 수 있도록 함.
    if 추천주소:
        # 주소에서 구나 군까지만 추출하여 필터링
        filtered_data = filtered_data[filtered_data['주소'].str.contains(추천주소)]

    recommended_data_list = []
    #빈리스트 생성 이 리스트는 추천된 장소들의 정보를 저장하는데 쓸거임
    
    for index, row in filtered_data.iterrows():
        # filtered_data에 있는 각 행을 하나씩 순회함.
        # iterrows() 함수는 데이터프레임의 각 행을 반복하면서 해당 인덱스와 행 데이터를 반환.
        lat1, lon1 = row['위도'], row['경도']
        # 해당 행의 '위도'와 '경도' 값을 가져와 lat1과 lon1 변수에 할당
        distances = []
        # distances라는 빈 리스트를 생성. 이 리스트는 현재 장소와 유사한 장소들의 거리 정보를 저장할 용도로 사용
        idx = data_recommend[data_recommend['제목'] == row['제목']].index[0]
        # data_recommend에서 현재 행의 '제목'과 일치하는 제목(장소)의 인덱스를 찾음. 
        # 이렇게 하여 현재 제목의 정보를 추후에 코사인 유사도 계산에서 사용할 수 있도록 가져옴.
        sim_scores = list(enumerate(cosine_sim[idx]))
        # cosine_sim 행렬에서 현재 장소와 다른 모든 장소 간의 코사인 유사도를 가져와 
        # enumerate()를 사용하여 인덱스와 함께 리스트로 변환.
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        #코사인 유사도 리스트를 유사도 값을 기준으로 내림차순 정렬
        place_index= [i[0] for i in sim_scores if i[0] != idx]  # 제목이 같은 장소는 제외
        # 현재 장소를 제외하고, 다른 장소들의 인덱스를 place_index에 저장 
        for i in place_index[:top_n]:
            # place_index에서 상위 top_n 개 만큼의 장소 인덱스를 하나씩 순회함.
            lat2, lon2 = data_recommend.iloc[i]['위도'], data_recommend.iloc[i]['경도']
            # data_recommend에서 해당 인덱스에 해당하는 장소의 '위도'와 '경도' 값을 가져와 lat2와 lon2 변수에 할당. 
            # 현재 검토 중인 다른 장소의 위치 정보를 나타낸다.
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            # haversine_distance 함수를 호출하여 현재 장소와 다른 장소 간의 거리를 계산, 거리 정보를 distance 변수에 저장
            if distance != 0 :
                distances.append((data_recommend.iloc[i]['제목'], data_recommend.iloc[i]['주소'], distance))
            else :
                pass
            # 만약 계산한 거리 값이 NaN이 아니면,distances 리스트에 현재 다른 영화의 '제목', '주소', 그리고 계산한 거리를 튜플 형태로 추가
            # 튜플 쓰는 이유 : 프로그램 돌아가는 도중에 리스트의 값이 변화되지 않아야하기 때문에
        distances.sort(key=lambda x: (x[2], -sim_scores[i][1]))
        # distances 리스트를 거리값과 코사인 유사도를 기준으로 정렬한다. 거리값은 작은값부터, 유사도는 큰값부터 정렬
        recommended_data_list.append([row['카테고리'], row['제목'], distances])
        # 해당 장소의 추천 및 거리 정보가 리스트에 담긴다.

    return recommended_data_list

# 특정 카테고리와 키워드, 주소를 기반으로 제목(장소) 추천 및 거리 계산 실행
추천카테고리 = ''  # None으로 두면 카테고리를 사용하지 않음
추천키워드 = ''  # None으로 두면 키워드를 사용하지 않음
추천주소 = '부평'  # None으로 두면 주소를 사용하지 않음
top_n = 3
recommended_distance = recommend_distance(data_recommend, 추천카테고리, 추천키워드, 추천주소, top_n)

for item in recommended_distance[:3]:
    카테고리, 제목, 거리정보 = item
    print(f"카테고리: {카테고리}, 제목: {제목}")
    for place, address, distance in 거리정보:
        print(f"  가까운 장소: {place}, 주소: {address}, 거리: {distance:.2f} km")

카테고리: 관광, 제목: 부평향교
  가까운 장소: 부평도호부관아, 주소: 인천 계양구 어사대로 20, 거리: 0.32 km
  가까운 장소: 자유공원, 주소: 인천 중구 자유공원남로 25, 거리: 11.26 km
  가까운 장소: 인천향교, 주소: 인천 미추홀구 매소홀로 589, 거리: 11.53 km
카테고리: 전시, 제목: 부평역사박물관
  가까운 장소: 영화공간주안, 주소: 인천 미추홀구 미추홀대로 716 메인프라자 7층, 거리: 7.59 km
  가까운 장소: 중앙어린이교통공원, 주소: 인천 미추홀구 문화로 22, 거리: 8.63 km
  가까운 장소: 인천광역시립박물관, 주소: 인천 연수구 청량로160번길 26, 거리: 12.62 km
카테고리: 쇼핑, 제목: 부평깡시장
  가까운 장소: 부평종합시장, 주소: 인천 부평구 부흥로316번길 38-3, 거리: 0.15 km
  가까운 장소: 신기시장, 주소: 인천 미추홀구 신기길58번길 6, 거리: 7.31 km
  가까운 장소: 용현시장, 주소: 인천 미추홀구 독정이로33번길 63 용현시장, 거리: 8.07 km


## TF-IDF 사용 키워드 리스트 뜨는버전

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import math

# 데이터프레임에서 가져오기 (예시 데이터로 대체)
# 데이터프레임에서 가져오기
data_recommend = data

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트', '위도', '경도', '주소']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))
tfidf_matrix = tfidf_vectorizer.fit_transform(data_for_recommend['combined'])

# 코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 위도 경도를 기반으로 두 지점 간의 거리를 계산하는 함수
def haversine_distance(lat1, lon1, lat2, lon2):
    radius = 6371.0  # 지구의 반지름 (단위: km)
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = radius * c
    return distance if not math.isnan(distance) else None

# 특정 카테고리와 키워드, 주소를 기반으로 제목(장소) 추천 및 거리 계산 실행
def recommend_distance(df, 추천카테고리=None, 추천키워드=None, 추천주소=None, top_n=3):
    filtered_data = df.copy()
    if 추천카테고리:
        filtered_data = filtered_data[filtered_data['카테고리'] == 추천카테고리]
    if 추천키워드:
        filtered_data = filtered_data[filtered_data['키워드리스트'].fillna('').str.contains(추천키워드)]
    if 추천주소:
        filtered_data = filtered_data[filtered_data['주소'].str.contains(추천주소)]

    recommended_data_list = []

    for index, row in filtered_data.iterrows():
        lat1, lon1 = row['위도'], row['경도']
        distances = []
        idx = data_recommend[data_recommend['제목'] == row['제목']].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        place_indices = [i[0] for i in sim_scores if i[0] != idx]
        for i in place_indices[:top_n]:
            lat2, lon2 = data_recommend.iloc[i]['위도'], data_recommend.iloc[i]['경도']
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            if distance is not None:
                distances.append((data_recommend.iloc[i]['제목'], data_recommend.iloc[i]['주소'], distance, data_recommend.iloc[i]['키워드리스트']))
        distances.sort(key=lambda x: (x[2], -sim_scores[i][1]))
        recommended_data_list.append([row['카테고리'], row['제목'], distances])

    return recommended_data_list

# 특정 카테고리와 키워드, 주소를 기반으로 제목(장소) 추천 및 거리 계산 실행
추천카테고리 = ''  # None으로 두면 카테고리를 사용하지 않음
추천키워드 = ''  # None으로 두면 키워드를 사용하지 않음
추천주소 = '부평'  # None으로 두면 주소를 사용하지 않음
top_n = 3
recommended_distance = recommend_distance(data_recommend, 추천카테고리, 추천키워드, 추천주소, top_n)

for item in recommended_distance[:3]:
    카테고리, 제목, 거리정보 = item
    print(f"카테고리: {카테고리}, 제목: {제목}")
    for place, address, distance, keywords in 거리정보:
        print(f"추천키워드: {keywords}, 가까운 장소: {place}, 주소: {address}, 거리: {distance:.2f} km")

카테고리: 관광, 제목: 부평향교
추천키워드: 부평도호부/272,관아/197,건물/90,부평/87,계양구/83,인천/82,은지/69,어사대/68,계양/67,역사/59,도호/48,은행나무/47,문화재/45,초등학교/42,정조/41,문화/40,해설/36,모습/35,청사/33,부사/31,인천광역시/30,지금/30,부평향교/29,계산동/26,연못/25,조선시대/25,여행/24,선정/24,관람/23,문화원/23,현재/22,위치/22,방문/22,나무/22,추정/21,규모/20,비석/20,시간/20,추가/19,당시/19,전시/19,놀이/19,복사/18,이웃/18,본문/18,기타/18,기능/18,설명/18,향교/18,대해/18,, 가까운 장소: 부평도호부관아, 주소: 인천 계양구 어사대로 20, 거리: 0.32 km
추천키워드: 공원/327,자유/302,인천/263,벚꽃/199,차이나타운/113,축제/86,중구/67,인천광역시/58,거리/57,주차장/52,사진/51,광주/50,여행/42,계단/41,제물포/39,생각/37,시간/37,구락부/37,사람/34,공간/33,카페/33,번길/32,나무/32,동상/31,주차/30,느낌/29,개항/29,맥아더/27,방문/27,오늘/26,바로/26,장군/24,근처/23,체험/23,정말/22,여기/22,시민/22,명소/22,이웃/21,추가/21,주말/21,하나/21,모습/21,지금/21,부스/21,복사/20,본문/20,기타/20,기능/20,구경/20,, 가까운 장소: 자유공원, 주소: 인천 중구 자유공원남로 25, 거리: 11.26 km
추천키워드: 인천/335,향교/249,관아/67,체험/57,교육/56,호부/54,아이/54,건물/45,미추홀/43,명륜당/38,대성전/37,시간/35,공간/35,행사/35,역사/32,기능/31,전통/27,인천광역시/25,여행/23,추가/22,조선시대/22,위패/22,놀토/22,문학동/21,도호/21,재현/21,제사/21,복사/20,이웃/20,본문/20,기타/20,준비/20,기관/20,추천/20,느낌/19,예절

## TF-IDF 사용 키워드리스트 뜨는버전(5개까지-실패)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import math

# 데이터프레임에서 가져오기 (예시 데이터로 대체)
# 데이터프레임에서 가져오기
data_recommend = data

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트', '위도', '경도', '주소']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3))
tfidf_matrix = tfidf_vectorizer.fit_transform(data_for_recommend['combined'])

# 코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 위도 경도를 기반으로 두 지점 간의 거리를 계산하는 함수
def haversine_distance(lat1, lon1, lat2, lon2):
    radius = 6371.0  # 지구의 반지름 (단위: km)
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = radius * c
    return distance if not math.isnan(distance) else None

# 특정 카테고리와 키워드, 주소를 기반으로 제목(장소) 추천 및 거리 계산 실행
def recommend_distance(df, 추천카테고리=None, 추천키워드=None, 추천주소=None, top_n=3):
    filtered_data = df.copy()
    if 추천카테고리:
        filtered_data = filtered_data[filtered_data['카테고리'] == 추천카테고리]
    if 추천키워드:
        filtered_data = filtered_data[filtered_data['키워드리스트'].fillna('').str.contains(추천키워드)]
    if 추천주소:
        filtered_data = filtered_data[filtered_data['주소'].str.contains(추천주소)]

    recommended_data_list = []

    for index, row in filtered_data.iterrows():
        lat1, lon1 = row['위도'], row['경도']
        distances = []
        idx = data_recommend[data_recommend['제목'] == row['제목']].index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        place_indices = [i[0] for i in sim_scores if i[0] != idx]
        for i in place_indices[:top_n]:
            lat2, lon2 = data_recommend.iloc[i]['위도'], data_recommend.iloc[i]['경도']
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            if distance is not None:
                keywords = data_recommend.iloc[i]['키워드리스트'].split(', ')[:5] if data_recommend.iloc[i]['키워드리스트'] else []
                distances.append((data_recommend.iloc[i]['제목'], data_recommend.iloc[i]['주소'], distance, keywords))
        distances.sort(key=lambda x: (x[2], -sim_scores[i][1]))
        recommended_data_list.append([row['카테고리'], row['제목'], distances])

    return recommended_data_list

# 특정 카테고리와 키워드, 주소를 기반으로 제목(장소) 추천 및 거리 계산 실행
추천카테고리 = ''  # None으로 두면 카테고리를 사용하지 않음
추천키워드 = ''  # None으로 두면 키워드를 사용하지 않음
추천주소 = '부평'  # None으로 두면 주소를 사용하지 않음
top_n = 3
recommended_distance = recommend_distance(data_recommend, 추천카테고리, 추천키워드, 추천주소, top_n)

for item in recommended_distance[:3]:
    카테고리, 제목, 거리정보 = item
    print(f"카테고리: {카테고리}, 제목: {제목}")
    for place, address, distance, keywords in 거리정보:
        keywords_str = ', '.join(keywords)
        print(f"키워드리스트: {keywords_str}, 가까운 장소: {place}, 주소: {address}, 거리: {distance:.2f} km")

카테고리: 관광, 제목: 부평향교
키워드리스트: 부평도호부/272,관아/197,건물/90,부평/87,계양구/83,인천/82,은지/69,어사대/68,계양/67,역사/59,도호/48,은행나무/47,문화재/45,초등학교/42,정조/41,문화/40,해설/36,모습/35,청사/33,부사/31,인천광역시/30,지금/30,부평향교/29,계산동/26,연못/25,조선시대/25,여행/24,선정/24,관람/23,문화원/23,현재/22,위치/22,방문/22,나무/22,추정/21,규모/20,비석/20,시간/20,추가/19,당시/19,전시/19,놀이/19,복사/18,이웃/18,본문/18,기타/18,기능/18,설명/18,향교/18,대해/18,, 가까운 장소: 부평도호부관아, 주소: 인천 계양구 어사대로 20, 거리: 0.32 km
키워드리스트: 공원/327,자유/302,인천/263,벚꽃/199,차이나타운/113,축제/86,중구/67,인천광역시/58,거리/57,주차장/52,사진/51,광주/50,여행/42,계단/41,제물포/39,생각/37,시간/37,구락부/37,사람/34,공간/33,카페/33,번길/32,나무/32,동상/31,주차/30,느낌/29,개항/29,맥아더/27,방문/27,오늘/26,바로/26,장군/24,근처/23,체험/23,정말/22,여기/22,시민/22,명소/22,이웃/21,추가/21,주말/21,하나/21,모습/21,지금/21,부스/21,복사/20,본문/20,기타/20,기능/20,구경/20,, 가까운 장소: 자유공원, 주소: 인천 중구 자유공원남로 25, 거리: 11.26 km
키워드리스트: 인천/335,향교/249,관아/67,체험/57,교육/56,호부/54,아이/54,건물/45,미추홀/43,명륜당/38,대성전/37,시간/35,공간/35,행사/35,역사/32,기능/31,전통/27,인천광역시/25,여행/23,추가/22,조선시대/22,위패/22,놀토/22,문학동/21,도호/21,재현/21,제사/21,복사/20,이웃/20,본문/20,기타/20,준비/20,기관/20,추천/20,느낌/19

## TF-IDF사용 키워드리스트뜨는 버전(카테고리도 잘뜨는거같음 5개까지 한줄한줄주석,2023-08-18준최종 )

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import math
## 라이브러리 임포트

# 데이터프레임에서 가져오기
data_recommend = data

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트', '위도', '경도', '주소']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체(전처리)
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기(분석할 텍스트 카테고리+키워드리스트)
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3)) # 텍스트데이터를 TF_IDF 행렬로 변환 ngram_range 파라미터를 통해서 구성 범위 설정
# "beautiful view"라는 문장이 있다면 ngram_range=(1, 3)의 경우 "beautiful", "view", "beautiful view" 등의 토큰이 생성
tfidf_matrix = tfidf_vectorizer.fit_transform(data_for_recommend['combined'])
# fit_transform 메서드를 사용하여 텍스트 데이터를 TF-IDF 행렬로 변환

# 코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 
# TF_IDF 행렬을 사용하여 코사인 유사도를 계산한다. 각 제목(장소) 간의 유사도(카테고리+키워드리스트 분석)를 나타내는 행렬

# 위도 경도를 기반으로 두 지점 간의 거리를 계산하는 함수
def haversine_distance(lat1, lon1, lat2, lon2):
    radius = 6371.0  # 지구의 반지름 (단위: km)

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c
    return distance if not math.isnan(distance) else None
#피타고라스의 정리와 유사함 a^2+c^2=b^2

# 특정 카테고리와 키워드, 주소를 기반으로 제목(장소) 추천 및 거리 계산 실행
def recommend_distance(df, 추천카테고리=None, 추천키워드=None, 추천주소=None, top_n=3):
    #DB의 데이터프레임을 받아서 특정 조건에 맞는 제목(장소)를 추천하고 그 제목(장소)들의 거리 정보를 함께 반환한다.
    #df=Db의 데이터프레임, 추천카테고리 : Df의 '카테고리', 추천키워드 : DF의 '키워드리스트', 추천주소 : Df의 제목(장소)의 주소 top_n: 개수
    filtered_data = df.copy()
    #df를 복사하여 filtererd_data 변수에 할당시킴 추천 조건을 적용하여 필터링한 데이터를 생성하면서, 원본데이터프레임은 그대로 유지
    if 추천카테고리:
        filtered_data = filtered_data[filtered_data['카테고리'] == 추천카테고리]
        #추천카테고리가 입력한 경우 (None이 아닌 경우), filtered_data를 해당 카테고리에 맞는 제목(장소)들로 필터링.
    if 추천키워드:
        # NaN 값을 빈 문자열로 대체한 후에 키워드를 포함하는지 확인
        filtered_data = filtered_data[filtered_data['키워드리스트'].fillna('').str.contains(추천키워드)]
        # 만약 추천키워드가 주어진 경우 (None이 아닌 경우), filtered_data를 해당 키워드가 포함된 제목들로 필터링
        # 이때 fillna('')를 사용하여 키워드가 NaN인 경우에도 필터링할 수 있도록 함.
    if 추천주소:
        # 주소에서 구나 군까지만 추출하여 필터링
        filtered_data = filtered_data[filtered_data['주소'].str.contains(추천주소)]

    recommended_data_list = []
    #빈리스트 생성 이 리스트는 추천된 장소들의 정보를 저장하는데 쓸거임
    
    for index, row in filtered_data.iterrows():
        #데이터프레임의 각 행을 하나씩 순회한다.
        lat1, lon1 = row['위도'], row['경도']
        # 현재 행의 위도,경도 값을 불러와서 lat1와 lon1에 할당시킴
        distances = []
        # 빈리스트 생성 현재 장소와 유사한 장소들의 거리정보를 저장할거임
        idx = data_recommend[data_recommend['제목'] == row['제목']].index[0]
        # data_recommend에서 현재 행의 '제목'과 일치하는 제목(장소)의 인덱스를 찾는다.
        sim_scores = list(enumerate(cosine_sim[idx]))
        # cosine_sim 행렬에서 현재 장소와 다른 모든 장소 간의 코사인 유사도를 가져와서 인덱스와 함께 리스트로 변환한다.
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        # 코사인 유사도를 유사도값 기준으로 내림차순 정렬한다.
        place_index = [i[0] for i in sim_scores if i[0] != idx]# 제목이 같은 장소는 제외
        # 현재 장소를 제외하고, 다른 장소들의 인덱스를 place_index에 저장 
        for i in place_index[:top_n]:
            # place_index에서 상위 top_n 개 만큼의 장소 인덱스를 하나씩 순회함.
            lat2, lon2 = data_recommend.iloc[i]['위도'], data_recommend.iloc[i]['경도']
            # data_recommend에서 해당 인덱스에 해당하는 장소의 '위도'와 '경도' 값을 가져와 lat2와 lon2 변수에 할당. 
            # 현재 검토 중인 다른 장소의 위치 정보를 나타낸다.
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            # haversine_distance 함수를 호출하여 현재 장소와 다른 장소 간의 거리를 계산, 거리 정보를 distance 변수에 저장
            if distance != 0  :
            # 현재 장소와 추천한 장소의 거리가 0이 아닌 경우에만 실행
                keywords = data_recommend.iloc[i]['키워드리스트'].split(',')[:5] if data_recommend.iloc[i]['키워드리스트'] else []
                # 현재 추천한 장소의"키워드 리스트의" 값을 가져와서 쉼표로 분리한다. 최대 5개까지의 키워드만 추출함.
                # 키워드 값이 없으면 else 써서 빈리스트로 출력되게
                distances.append((data_recommend.iloc[i]['카테고리'], data_recommend.iloc[i]['제목'], data_recommend.iloc[i]['주소'], distance, keywords))
                # 'distances' 리스트에 추천장소의 ;'제목','주소','거리', '추천키워드','카테고리'를 튜플형태로 추가한다.
                # 튜플 쓰는 이유 : 프로그램 돌아가는 도중에 리스트의 값이 변화되지 않아야하기 때문에
            else :
                pass
            #이 경우가 아니면 pass해서 넘어가게
        distances.sort(key=lambda x: (x[2], -sim_scores[i][1]))
        # distances의 리스트를 정렬한다. 정렬기준 두가지
        # 거리값을 오름차순으로 정렬
        # 코사인유사도값의 음수값을 내림차순으로 정렬함. 코사인유사도는 음수값으로 갈수록 유사도 높은거같음 정확히모름
        recommended_data_list.append([row['카테고리'], row['제목'], distances])
        # recommended_data_list 리스트에 현재 장소의 추천 카테고리, 제목, 거리정보, 추천키워드가 포함된 distances 리스트를 추가한다.

    return recommended_data_list

# 특정 카테고리와 키워드, 주소를 기반으로 제목(장소) 추천 및 거리 계산 실행
추천카테고리 = '음식'  # None으로 두면 카테고리를 사용하지 않음
추천키워드 = '맛집'  # None으로 두면 키워드를 사용하지 않음
추천주소 = ''  # None으로 두면 주소를 사용하지 않음
top_n = 3
recommended_distance = recommend_distance(data_recommend, 추천카테고리, 추천키워드, 추천주소, top_n)

for item in recommended_distance[:3]:
    카테고리, 제목, 거리정보 = item
    print(f"카테고리: {카테고리}, 추천장소: {제목}, 주소: {address}\n")
    for category, place, address, distance, keywords in 거리정보:
        keywords_str = ', '.join(keywords)
        print(f"카테고리 : {category},가까운 추천 장소: {place}, \n주소: {address}, \n거리: {distance:.2f} km, \n추천키워드: {keywords_str}\n")
    

카테고리: 음식, 추천장소: 돈비어천가, 주소: 인천 연수구 송도과학로16번길 33-3

카테고리 : 음식,가까운 추천 장소: 우미가든, 
주소: 인천 남동구 남동대로916번길 54, 
거리: 7.37 km, 
추천키워드: 우미/100, 가든/90, 맛집/57, 간석동/51, 고기/40

카테고리 : 쇼핑,가까운 추천 장소: 열우물전통시장(구 십정종합시장), 
주소: 인천 부평구 배곶남로21번길 18, 
거리: 5.91 km, 
추천키워드: 시장/428, 우물/126, 전통/115, 십정/105, 부평/85

카테고리 : 음식,가까운 추천 장소: 굴포먹거리타운, 
주소: 인천광역시 남동구 인하로, 
거리: 7.94 km, 
추천키워드: 맛집/117, 굴포/99, 먹거리/97, 족발/97, 타운/96

카테고리: 음식, 추천장소: 명품삼계탕, 주소: 인천광역시 남동구 인하로

카테고리 : 음식,가까운 추천 장소: 향계, 
주소: 인천 남동구 인주대로 664, 
거리: 5.67 km, 
추천키워드: 백숙/165, 능이버섯/161, 향계/111, 맛집/81, 오리/80

카테고리 : 음식,가까운 추천 장소: 풍전식당, 
주소: 인천 미추홀구 제일로 41, 
거리: 2.30 km, 
추천키워드: 식당/213, 육개장/180, 해장국/120, 인천/73, 맛집/69

카테고리 : 음식,가까운 추천 장소: 명가한방삼계탕, 
주소: 인천 부평구 부평대로17번길 21, 
거리: 7.01 km, 
추천키워드: 삼계탕/315, 명가/106, 맛집/86, 부평/71, 반찬/38

카테고리: 음식, 추천장소: 바다해물탕, 주소: 인천 부평구 부평대로17번길 21

카테고리 : 음식,가까운 추천 장소: 우리해물탕, 
주소: 인천 남동구 담방로48번길 3, 
거리: 1.11 km, 
추천키워드: 해물탕/256, 맛집/122, 우리/55, 해물/45, 국물/43

카테고리 : 음식,가까운 추천 장소: 소금빛풍천장어구이, 
주소: 인천 부평구 길주로547번길 5, 
거리: 6.18 km,

## TF-IDF 사용 카테고리 다르게 뜨게해보기(2023-08-18)

In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import math
## 라이브러리 임포트

# 데이터프레임에서 가져오기
data_recommend = data

# 데이터프레임을 카테고리와 키워드 컬럼만 남기고 복사
data_for_recommend = data_recommend[['카테고리', '키워드리스트', '위도', '경도', '주소']].copy()

# 각 컬럼의 NaN 값을 빈 문자열로 대체(전처리)
data_for_recommend.fillna('', inplace=True)

# 텍스트 데이터 합치기(분석할 텍스트 카테고리+키워드리스트)
data_for_recommend['combined'] = data_for_recommend['카테고리'] + ' ' + data_for_recommend['키워드리스트']

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3)) # 텍스트데이터를 TF_IDF 행렬로 변환 ngram_range 파라미터를 통해서 구성 범위 설정
# "beautiful view"라는 문장이 있다면 ngram_range=(1, 3)의 경우 "beautiful", "view", "beautiful view" 등의 토큰이 생성
tfidf_matrix = tfidf_vectorizer.fit_transform(data_for_recommend['combined'])
# fit_transform 메서드를 사용하여 텍스트 데이터를 TF-IDF 행렬로 변환

# 코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) 
# TF_IDF 행렬을 사용하여 코사인 유사도를 계산한다. 각 제목(장소) 간의 유사도(카테고리+키워드리스트 분석)를 나타내는 행렬

# 위도 경도를 기반으로 두 지점 간의 거리를 계산하는 함수
def haversine_distance(lat1, lon1, lat2, lon2):
    radius = 6371.0  # 지구의 반지름 (단위: km)

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c
    return distance if not math.isnan(distance) else None
#피타고라스의 정리와 유사함 a^2+c^2=b^2

# 특정 카테고리와 키워드, 주소를 기반으로 제목(장소) 추천 및 거리 계산 실행
def recommend_distance(df, 추천카테고리=None, 추천키워드=None, 추천주소=None, top_n=3):
    #DB의 데이터프레임을 받아서 특정 조건에 맞는 제목(장소)를 추천하고 그 제목(장소)들의 거리 정보를 함께 반환한다.
    #df=Db의 데이터프레임, 추천카테고리 : Df의 '카테고리', 추천키워드 : DF의 '키워드리스트', 추천주소 : Df의 제목(장소)의 주소 top_n: 개수
    filtered_data = df.copy()
    #df를 복사하여 filtererd_data 변수에 할당시킴 추천 조건을 적용하여 필터링한 데이터를 생성하면서, 원본데이터프레임은 그대로 유지
    if 추천카테고리:
        filtered_data = filtered_data[filtered_data['카테고리'] == 추천카테고리]
        #추천카테고리가 입력한 경우 (None이 아닌 경우), filtered_data를 해당 카테고리에 맞는 제목(장소)들로 필터링.
    if 추천키워드:
        # NaN 값을 빈 문자열로 대체한 후에 키워드를 포함하는지 확인
        filtered_data = filtered_data[filtered_data['키워드리스트'].fillna('').str.contains(추천키워드)]
        # 만약 추천키워드가 주어진 경우 (None이 아닌 경우), filtered_data를 해당 키워드가 포함된 제목들로 필터링
        # 이때 fillna('')를 사용하여 키워드가 NaN인 경우에도 필터링할 수 있도록 함.
    if 추천주소:
        # 주소에서 구나 군까지만 추출하여 필터링
        filtered_data = filtered_data[filtered_data['주소'].str.contains(추천주소)]

    recommended_data_list = []
    #빈리스트 생성 이 리스트는 추천된 장소들의 정보를 저장하는데 쓸거임
    
    for index, row in filtered_data.iterrows():
        #데이터프레임의 각 행을 하나씩 순회한다.
        lat1, lon1 = row['위도'], row['경도']
        # 현재 행의 위도,경도 값을 불러와서 lat1와 lon1에 할당시킴
        distances = []
        # 빈리스트 생성 현재 장소와 유사한 장소들의 거리정보를 저장할거임
        idx = data_recommend[data_recommend['제목'] == row['제목']].index[0]
        # data_recommend에서 현재 행의 '제목'과 일치하는 제목(장소)의 인덱스를 찾는다.
        sim_scores = list(enumerate(cosine_sim[idx]))
        # cosine_sim 행렬에서 현재 장소와 다른 모든 장소 간의 코사인 유사도를 가져와서 인덱스와 함께 리스트로 변환한다.
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        # 코사인 유사도를 유사도값 기준으로 내림차순 정렬한다.
        place_index = [i[0] for i in sim_scores if i[0] != idx]# 제목이 같은 장소는 제외
        # 현재 장소를 제외하고, 다른 장소들의 인덱스를 place_index에 저장 
        for i in place_index[:top_n]:
            # place_index에서 상위 top_n 개 만큼의 장소 인덱스를 하나씩 순회함.
            lat2, lon2 = data_recommend.iloc[i]['위도'], data_recommend.iloc[i]['경도']
            # data_recommend에서 해당 인덱스에 해당하는 장소의 '위도'와 '경도' 값을 가져와 lat2와 lon2 변수에 할당. 
            # 현재 검토 중인 다른 장소의 위치 정보를 나타낸다.
            distance = haversine_distance(lat1, lon1, lat2, lon2)
            # haversine_distance 함수를 호출하여 현재 장소와 다른 장소 간의 거리를 계산, 거리 정보를 distance 변수에 저장
            if distance != 0 and distance<20 :
            # 현재 장소와 추천한 장소의 거리가 0이 아닌 경우에만 실행
                keywords = data_recommend.iloc[i]['키워드리스트'].split(',')[:5] if data_recommend.iloc[i]['키워드리스트'] else []
                # 현재 추천한 장소의"키워드 리스트의" 값을 가져와서 쉼표로 분리한다. 최대 5개까지의 키워드만 추출함.
                # 키워드 값이 없으면 else 써서 빈리스트로 출력되게
                distances.append((data_recommend.iloc[i]['카테고리'], data_recommend.iloc[i]['제목'], data_recommend.iloc[i]['주소'], distance, keywords))
                # 'distances' 리스트에 추천장소의 ;'제목','주소','거리', '추천키워드','카테고리'를 튜플형태로 추가한다.
                # 튜플 쓰는 이유 : 프로그램 돌아가는 도중에 리스트의 값이 변화되지 않아야하기 때문에
            else :
                pass
            #이 경우가 아니면 pass해서 넘어가게
        distances.sort(key=lambda x: (x[2], -sim_scores[i][1]))
        # distances의 리스트를 정렬한다. 정렬기준 두가지
        # 거리값을 오름차순으로 정렬
        # 코사인유사도값의 음수값을 내림차순으로 정렬함. 코사인유사도는 음수값으로 갈수록 유사도 높은거같음 정확히모름
        recommended_data_list.append([row['카테고리'], row['제목'], distances])
        # recommended_data_list 리스트에 현재 장소의 추천 카테고리, 제목, 거리정보, 추천키워드가 포함된 distances 리스트를 추가한다.

    return recommended_data_list

# 특정 카테고리와 키워드, 주소를 기반으로 제목(장소) 추천 및 거리 계산 실행
추천카테고리 = ''  # None으로 두면 카테고리를 사용하지 않음
추천키워드 = ''  # None으로 두면 키워드를 사용하지 않음
추천주소 = '부평구'  # None으로 두면 주소를 사용하지 않음
top_n = 3
recommended_distance = recommend_distance(data_recommend, 추천카테고리, 추친키워드, 추천주소, top_n)

for item in recommended_distance[:3]:
    카테고리, 제목, 거리정보 = item
    print(f"카테고리: {카테고리}, 추천장소: {제목}\n")
    for category, place, address, distance, keywords in 거리정보:
        keywords_str = ', '.join(keywords)
        print(f"카테고리 : {category},가까운 추천 장소: {place}, \n주소: {address}, \n거리: {distance:.2f} km, \n추천키워드: {keywords_str}\n")
        

NameError: name '추친키워드' is not defined